<a href="https://colab.research.google.com/github/akajammythakkar/rag-with-gemini/blob/main/RAG_with_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install langchain chromadb pypdf google-generativeai sentence_transformers Ipython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl

#### Import necessary libraries

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from pypdf import PdfReader
import google.generativeai as genai
from pprint import pprint
from google.colab import userdata
from IPython.display import Markdown, display

In [4]:
# Create a PdfReader object to read the PDF file
reader = PdfReader("/content/Alphabet Anual Report.pdf")

# Extract text from each page in the PDF and strip any leading/trailing whitespace
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter out any empty strings from the extracted texts
pdf_texts = [text for text in pdf_texts if text]

# Pretty-print the text from the first page of the PDF
pprint(pdf_texts[0])

('UNITED STATES\n'
 'SECURITIES AND EXCHANGE COMMISSION\n'
 'Washington, D.C. 20549\n'
 '___________________________________________\n'
 'FORM 10-K\n'
 '___________________________________________\n'
 '(Mark One)\n'
 '☒ ANNUAL REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE '
 'ACT OF 1934\n'
 'For the fiscal year ended December 31, 2023\n'
 'OR\n'
 '☐ TRANSITION REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES '
 'EXCHANGE ACT OF 1934\n'
 'For the transition period from              to             .\n'
 'Commission file number: 001-37580\n'
 '___________________________________________\n'
 'Alphabet Inc.\n'
 '(Exact name of registrant as specified in its charter)\n'
 '___________________________________________\n'
 'Delaware 61-1767919\n'
 '(State or other jurisdiction of incorporation or organization) (I.R.S. '
 'Employer Identification No.)\n'
 '1600 Amphitheatre Parkway\n'
 'Mountain V iew, CA 94043\n'
 '(Address of principal executive offices, including

In [5]:
# Create a RecursiveCharacterTextSplitter object with specified separators, chunk size, and chunk overlap
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],  # List of separators for splitting the text
    chunk_size=1000,  # Maximum size of each text chunk
    chunk_overlap=0  # Number of characters to overlap between chunks
)

# Join the extracted PDF texts with '\n\n' and split the combined text into chunks
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

# Pretty-print the text of the 11th chunk (index 10) of the split text
pprint(character_split_texts[10])

# Print the total number of chunks created
print(f"\nTotal chunks: {len(character_split_texts)}")

('•our expectation that our monetization trends will fluctuate, which could '
 'affect our revenues and margins;\n'
 '•fluctuations in paid clicks and cost-per-click as well as impressions and '
 'cost-per-impression, and various\n'
 'factors contributing to such fluctuations;\n'
 '•our expectation that we will continue to periodically review, refine, and '
 'update our methodologies for\n'
 'monitoring, gathering, and counting the number of paid clicks and '
 'impressions;\n'
 '•our expectation that our results will be affected by our performance in '
 'international markets as users in\n'
 'developing economies increasingly come online;\n'
 '•our expectation that our foreign exchange risk management program will not '
 'fully offset our net exposure to\n'
 'fluctuations in foreign currency exchange rates;\n'
 '•the expected variability of gains and losses related to hedging activities '
 'under our foreign exchange risk\n'
 'management program;\n'
 '•the amount and timing of revenue 

In [6]:
# Create a SentenceTransformersTokenTextSplitter object with specified chunk overlap and tokens per chunk
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

# Initialize an empty list to hold the token-split texts
token_split_texts = []

# Loop through each chunk in the character-split texts
for text in character_split_texts:
    # Split the text into smaller chunks using the token splitter and add them to the token_split_texts list
    token_split_texts += token_splitter.split_text(text)

# Print the wrapped text of the 11th chunk (index 10) of the token-split text
pprint(token_split_texts[10])

# Print the total number of token-split chunks created
print(f"\nTotal chunks: {len(token_split_texts)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

('table of contents alphabet inc. note about forward - looking statements this '
 'annual report on form 10 - k contains forward - looking statements within '
 'the meaning of the private securities litigation reform act of 1995. these '
 'include, among other things, statements regarding : • the growth of our '
 'business and revenues and our expectations about the factors that influence '
 'our success and trends in our business ; • fluctuations in our revenues and '
 'margins and various factors contributing to such fluctuations ; • our '
 'expectation that the continuing shift from an offline to online world will '
 'continue to benefit our business ; • our expectation that the portion of our '
 'revenues that we derive beyond advertising will continue to increase and may '
 'affect our margins ; • our expectation that our traffic acquisition costs ( '
 'tac ) and the associated tac rate will fluctuate, which could affect our '
 'overall margins ;')

Total chunks: 468


In [7]:
# Create a SentenceTransformerEmbeddingFunction object
embedding_function = SentenceTransformerEmbeddingFunction()

# Generate embeddings for the 11th chunk (index 10) of the token-split text and print the result
print(embedding_function([token_split_texts[10]]))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.03815276175737381, -0.028939982876181602, 0.010575269348919392, -0.04519069194793701, 0.024973738938570023, 0.07247766852378845, 0.04143134504556656, 0.03898466378450394, 0.05970046669244766, 0.028211917728185654, 0.02014230191707611, 0.1360497623682022, -0.02045018970966339, -0.06197531893849373, 0.00901737343519926, -0.010835169814527035, 0.034079067409038544, 0.0397229939699173, -0.028853977099061012, 0.027246493846178055, -0.018192622810602188, 0.046652358025312424, 0.01505122147500515, 0.0331496000289917, -0.032329946756362915, -0.030759112909436226, -0.08560162037611008, 0.023434406146407127, -0.04526044800877571, -0.06211463734507561, -0.07926011830568314, 0.037720490247011185, 0.0325380340218544, 0.01959894597530365, 0.040424589067697525, -0.07005400210618973, -0.0181980449706316, -0.015993181616067886, 0.0392431914806366, 0.003032135544344783, 0.02793501317501068, -0.11963129788637161, -0.0568445585668087, 0.00921389739960432, 0.018856106325984, 0.00542488181963563, -0.00

In [8]:
# Create a ChromaDB client
chroma_client = chromadb.Client()

# Create a new collection in ChromaDB with the name "Alphabet Annual Report" and the specified embedding function
chroma_collection = chroma_client.create_collection("alphabet_annual_report", embedding_function=embedding_function)

# Generate a list of string IDs corresponding to the number of token-split text chunks
ids = [str(i) for i in range(len(token_split_texts))]

# Add the token-split text chunks to the ChromaDB collection using the generated IDs
chroma_collection.add(ids=ids, documents=token_split_texts)

# Count and return the number of documents in the ChromaDB collection
chroma_collection.count()

468

In [10]:
# Step 1: Retrieve the API key from user data
GEMINI_API_KEY = userdata.get('API_KEY')  # Get API Key from Secrets

# Step 2: Configure the GenAI client with the retrieved API key
genai.configure(api_key=GEMINI_API_KEY)

# Step 3: Define the generation configuration for the model
generation_config = {
    "temperature": 0.9,       # Controls the randomness of the output (higher values mean more random)
    "top_p": 1,               # Controls nucleus sampling (1 means no filtering)
    "top_k": 1,               # Controls the number of highest probability tokens to consider (1 means only the highest)
    "max_output_tokens": 2048 # Maximum number of tokens in the output
}

# Step 4: Initialize the generative model with the specified name and configuration
model = genai.GenerativeModel(
    model_name="gemini-1.0-pro",       # Name of the model
    generation_config=generation_config  # Configuration for text generation
)

In [11]:
def rag(query, retrieved_documents):
    # Combine the retrieved documents into a single string, separated by double newlines
    information = "\n\n".join(retrieved_documents)

    # Create the message for the generative model, providing context and the user's query
    messages = [
        "You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
        f"Question: {query}. \n Information: {information}"
    ]

    # Generate a response using the configured generative model
    response = model.generate_content(messages)

    # Return the text part of the first candidate's response
    return response.candidates[0].content.parts[0].text

In [12]:
# Step 1: Define the query string
query = "What are some major revenues coming from?"

# Step 2: Query the ChromaDB collection with the specified query string, retrieving the top 3 results
results = chroma_collection.query(query_texts=[query], n_results=3)

# Step 3: Extract the list of retrieved documents from the query results
retrieved_documents = results['documents'][0]

# Step 4: Loop through each retrieved document, print the wrapped text, and add a newline for readability
for document in retrieved_documents:
    pprint(document)
    print('\n')


('table of contents alphabet inc. note 2. revenues disaggregated revenues the '
 'following table presents revenues disaggregated by type ( in millions ) : '
 'year ended december 31, 2021 2022 2023 google search & other $ 148, 951 $ '
 '162, 450 $ 175, 033 youtube ads 28, 845 29, 243 31, 510 google network 31, '
 '701 32, 780 31, 312 google advertising 209, 497 224, 473 237, 855 google '
 'subscriptions, platforms, and devices 28, 032 29, 055 34, 688 google '
 'services total 237, 529 253, 528 272, 543 google cloud 19, 206 26, 280 33, '
 '088 other bets 753 1, 068 1, 527 hedging gains ( losses ) 149 1, 960 236 '
 'total revenues $ 257, 637 $ 282, 836 $ 307, 394 no individual customer or '
 'groups of affiliated customers represented more than 10 % of our revenues in '
 '2021, 2022, or 2023. the following table presents revenues disaggregated by '
 'geography, based on the addresses of our customers ( in millions ) : year '
 'ended december 31, 2021 2022 2023 united states $ 117, 854 4

In [14]:
# Generate the response using the RAG function with the provided query and retrieved documents
output = rag(query=query, retrieved_documents=retrieved_documents)

# Print the generated response
display(Markdown(output))

- Google Search & other
- Youtube ads
- Google advertising
- Google subscriptions, platforms, and devices
- Google Cloud
- Other bets